## Attractor Simulation

Please refer to the Method section

In [1]:
import numpy as np
import pandas as pd
import itertools
import networkx as nx
import copy
import os

from pyboolnet.file_exchange import bnet2primes, primes2bnet
from pyboolnet.interaction_graphs import primes2igraph
from pyboolnet.state_transition_graphs import primes2stg
from pyboolnet.attractors import compute_attractors_tarjan

from modules.attractorSim import rand_initial_states, compute_attractor_from_primes, compute_phenotype, Simulation
from modules.stability import compute_networkStability

In [2]:
network_dir = './network/'
model_file = network_dir + 'toy_Network.bnet'
primes = bnet2primes(model_file)
nodeList = list(primes.keys())
graph = primes2igraph(primes)
update_mode = "synchronous"

phenotype = {'P1':{'Output1':1, 'Output2':0}, 'P2':{'Output1':0, 'Output2':1}, 'P3':{'Output1':1, 'Output2':1}, 'P4':{'Output1':0, 'Output2':0}}
phenotypeAnnot = {'P1':-1,'P2':1, 'P3':0, 'P4':0}
markers = ['Output1','Output2']


if 2**len(nodeList) >= 100000: num_init = 100000
else:  num_init = 2**len(nodeList)
initState = rand_initial_states(num_init, len(nodeList))

In [3]:
save_dir = './result/' 
if save_dir.split('/')[1] not in os.listdir(): os.mkdir(save_dir)
save_perturbname = save_dir + 'toy_single_simul_result.csv'

## Node perturbation analysis

In [4]:
fix_dict = {'Input':0}
allsingles = [{n:1} for n in nodeList] + [{n:0} for n in nodeList]
#allsingles

In [5]:
perturb_p = pd.DataFrame([]) # average activities of the marker nodes
perturb_s = pd.DataFrame([]) # network stabiltiy 
for perturb in allsingles:
    fix_dict_tmp = copy.deepcopy(fix_dict)
    fix_dict_tmp.update(perturb)
    print(fix_dict_tmp)
    primes_new, pheno_df, att_ave_pd, attrs_dict = Simulation(fix_dict_tmp, primes, update_mode, initState, phenotype, phenotypeAnnot)
    dT = pd.DataFrame([0 for _ in markers], index = markers, columns = [str(list(perturb.items()))])
    dT.loc[markers,:] = att_ave_pd.loc[markers,:].values
    perturb_p = pd.concat([perturb_p,dT],axis=1)     
    dS = pd.DataFrame.from_dict(compute_networkStability(attrs_dict, graph, nodeList)[1], orient='index', columns = [str(list(perturb.items()))] )
    perturb_s = pd.concat([perturb_s,dS],axis=1)    
perturb_result = pd.concat([perturb_p, perturb_s]).T        
perturb_result.to_csv(save_perturbname)

{'Input': 1}
Attractor simulation time : 0.11492514610290527
           Ratio
phenotype       
P1           1.0
{'Input': 0, 'Output1': 1}
Attractor simulation time : 0.15722179412841797
            Ratio
phenotype        
P1         0.1875
P3         0.8125
{'Input': 0, 'Output2': 1}
Attractor simulation time : 0.16115689277648926
              Ratio
phenotype          
P2         0.359375
P3         0.640625
{'Input': 0, 'n2': 1}
Attractor simulation time : 0.10706901550292969
           Ratio
phenotype       
P1           1.0
{'Input': 0, 'n3': 1}
Attractor simulation time : 0.1586155891418457
           Ratio
phenotype       
P2           1.0
{'Input': 0, 'n4': 1}
Attractor simulation time : 0.15333795547485352
            Ratio
phenotype        
P1         0.1875
P3         0.8125
{'Input': 0, 'n5': 1}
Attractor simulation time : 0.1274271011352539
           Ratio
phenotype       
P2           1.0
{'Input': 0, 'n6': 1}
Attractor simulation time : 0.1275467872619629
           Rat

In [7]:
perturb_p # average activities of the marker nodes

,"[('Input', 1)]","[('Output1', 1)]","[('Output2', 1)]","[('n2', 1)]","[('n3', 1)]","[('n4', 1)]","[('n5', 1)]","[('n6', 1)]","[('n7', 1)]","[('n8', 1)]",...,"[('Output1', 0)]","[('Output2', 0)]","[('n2', 0)]","[('n3', 0)]","[('n4', 0)]","[('n5', 0)]","[('n6', 0)]","[('n7', 0)]","[('n8', 0)]","[('n9', 0)]"
Output1,1.0,1.000000,0.414062,1.0,0.0,1.0000,0.0,0.0,0.414062,0.414062,...,0.000000,0.414062,0.414062,1.0,0.0000,1.0,1.0,0.414062,0.414062,0.414062
Output2,0.0,0.585938,1.000000,0.0,1.0,0.5625,1.0,1.0,0.585938,0.585938,...,0.585938,0.000000,0.585938,0.0,0.8125,0.0,0.0,0.585938,0.585938,0.585938


In [8]:
perturb_s.loc['F_major_att',:] # frustration of major attractor

[('Input', 1)]     -13.0
[('Output1', 1)]     8.0
[('Output2', 1)]     7.0
[('n2', 1)]         -5.0
[('n3', 1)]        -14.0
[('n4', 1)]          6.0
[('n5', 1)]        -12.0
[('n6', 1)]        -14.0
[('n7', 1)]         10.0
[('n8', 1)]          8.0
[('n9', 1)]          4.0
[('Input', 0)]       9.0
[('Output1', 0)]     8.0
[('Output2', 0)]     7.0
[('n2', 0)]          9.0
[('n3', 0)]         -3.0
[('n4', 0)]        -12.0
[('n5', 0)]        -10.0
[('n6', 0)]        -10.0
[('n7', 0)]          8.0
[('n8', 0)]          4.0
[('n9', 0)]          0.0
Name: F_major_att, dtype: float64